In [3]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import * 
from ydata_profiling import ProfileReport
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt

# Get the data and create the output directory (Extraction)

In [4]:
path_dir_data = "/home/ale1726/proyects/datalake/clientes/data/productos/SIPE/data"

In [5]:
# Obtenemos la fecha actual para crear un directorio
# donde se guardaran los archivos de salida
date_now = datetime.now().strftime("%d_%m_%Y") 
path_dir_today =  os.path.join(path_dir_data, date_now)

# Creamos el directorio si no existe
os.makedirs(path_dir_today, exist_ok=True)

In [6]:
query_productos_sipe = """ 
WITH CLIENTES_ACTIVOS AS (
    SELECT  BCO_CLAVE_FFON, BCO_CLAVE, BCO_NOMBRE, SPB.SUC_NUMERO
    FROM SIPE.SIPE_BANCOS sp
  	LEFT JOIN SIPE.SIPE_SUCURSALES_BANCO SPB ON SP.BCO_CLAVE = SPB.SUC_BCO_CLAVE
    WHERE SP.BCO_ESTATUS = 'A'
), CLI_CUENTAS_CONTRAPARTE AS (
	SELECT  CA.*, SCC.CIP_DIV_CLAVE, SCC.CIP_TES_CLAVE, SCC.CIP_SUC_NUMERO
	FROM CLIENTES_ACTIVOS CA
	LEFT JOIN SIPE.SIPE_CUENTAS_CONTRAPARTE SCC ON CA.SUC_NUMERO = SCC.CIP_SUC_NUMERO
), CLI_INV_PRES AS (
	SELECT CCC.*, SIP.*
	FROM CLI_CUENTAS_CONTRAPARTE CCC
	LEFT JOIN SIPE.SIPE_INVERSIONES_PRESTAMOS SIP ON ( SIP.IPE_SUC_NUMERO = CCC.CIP_SUC_NUMERO AND SIP.IPE_TES_CLAVE = CCC.CIP_TES_CLAVE AND SIP.IPE_DIV_CLAVE = CCC.CIP_DIV_CLAVE )
) SELECT * FROM CLI_INV_PRES
"""

In [7]:
path_dir_productos = get_table(path_dir_today, db_sipe, "productos_clientes_sipe", query = query_productos_sipe)

# Transform the data and output the transformation

In [8]:
path_dir_output_transformation = os.path.join("/home/ale1726/proyects/datalake/clientes/data/productos/SIPE/agrupados", date_now)
os.makedirs(path_dir_output_transformation, exist_ok=True)
path_dir_output_transformation

'/home/ale1726/proyects/datalake/clientes/data/productos/SIPE/agrupados/22_04_2025'

In [9]:
productos_sipe =  pd.read_csv(path_dir_productos, low_memory=False)

In [10]:
productos_sipe

,BCO_CLAVE_FFON,BCO_CLAVE,BCO_NOMBRE,SUC_NUMERO,CIP_DIV_CLAVE,CIP_TES_CLAVE,CIP_SUC_NUMERO,IPE_FOLIO,IPE_FECHA_REGISTRO,IPE_FECHA_MODIF,...,IPE_TES_CLAVE,IPE_SUC_NUMERO,IPE_FECHA_VALOR,IPE_IMPORTE,IPE_PLAZO,IPE_TASA,IPE_FECHA_VENCIMIENTO,IPE_FOLIO_ORIGEN,IPE_FECHA_CONCERTACION,IPE_EVENTO
0,BOTKUS33,1,BANK OF TOKYO-MITSUBISHI,1.0,USD,MAT,1.0,69.0,1997-02-07 00:00:00,1997-02-10 00:00:00,...,MAT,1.0,1997-02-07 00:00:00,100000000.0,3.0,5.0000,1997-02-10 00:00:00,NaN,NaN,NaN
1,NaN,6,BLADEX,6.0,USD,MAT,6.0,70.0,1997-02-07 00:00:00,1997-02-14 00:00:00,...,MAT,6.0,1997-02-07 00:00:00,10000000.0,7.0,5.4375,1997-02-14 00:00:00,NaN,NaN,NaN
2,BNPAUS3N,2,"BNP-PARIBAS, N.Y.",2.0,USD,MAT,2.0,71.0,1997-02-07 00:00:00,1997-02-10 00:00:00,...,MAT,2.0,1997-02-07 00:00:00,300000000.0,3.0,5.1250,1997-02-10 00:00:00,NaN,NaN,NaN
3,NaN,4,BANK OF MONTREAL,4.0,USD,MAT,4.0,72.0,1997-02-07 00:00:00,1997-02-10 00:00:00,...,MAT,4.0,1997-02-07 00:00:00,250000000.0,3.0,5.1250,1997-02-10 00:00:00,NaN,NaN,NaN
4,NaN,3,DANSKE BANK,3.0,USD,MAT,3.0,73.0,1997-02-07 00:00:00,1997-02-10 00:00:00,...,MAT,3.0,1997-02-07 00:00:00,300000000.0,3.0,5.1250,1997-02-10 00:00:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67044,NaN,289,"CREDIT INDUSTRIEL ET COMMERCIAL,NY",289.0,USD,MAT,289.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67045,NaN,251,"HSBC BANK, USA",251.0,USD,GCM,251.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67046,NaN,12,BANCO ANAHUAC,12.0,USD,GCM,12.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67047,NaN,4,BANK OF MONTREAL,4.0,USD,MAN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
productos_sipe["IPE_FECHA_REGISTRO"] = pd.to_datetime(productos_sipe["IPE_FECHA_REGISTRO"])
productos_sipe["AÑO_FOPE"] = productos_sipe["IPE_FECHA_REGISTRO"].dt.year
productos_sipe["MES_FOPE"] = productos_sipe["IPE_FECHA_REGISTRO"].dt.month
productos_sipe["MES_STR"] = productos_sipe["IPE_FECHA_REGISTRO"].dt.month_name()  

In [12]:
agrupado_operaciones_anuales = (
    productos_sipe.groupby(['AÑO_FOPE'], as_index=False)
    .agg(
        num_operaciones=('IPE_FOLIO', 'count'), 
        monto_anuales_sum=('IPE_IMPORTE', 'sum'), 
        monto_anuales_min=('IPE_IMPORTE', 'min'), 
        monto_anuales_max=('IPE_IMPORTE', 'max'), 
        monto_anuales_mean=('IPE_IMPORTE', 'mean'), 
        monto_anuales_median=('IPE_IMPORTE', 'median')
    ).sort_values(by=['AÑO_FOPE'], ascending=True)
)

In [13]:
agrupado_operaciones_anuales.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales.csv"), index=False)

In [14]:
agrupado_operaciones_anuales_mes = (
    productos_sipe.groupby(['AÑO_FOPE','MES_FOPE', 'MES_STR'], as_index=False)
    .agg(
        num_operaciones=('IPE_FOLIO', 'count'), 
        monto_anuales_sum=('IPE_IMPORTE', 'sum'), 
        monto_anuales_min=('IPE_IMPORTE', 'min'), 
        monto_anuales_max=('IPE_IMPORTE', 'max'), 
        monto_anuales_mean=('IPE_IMPORTE', 'mean'), 
        monto_anuales_median=('IPE_IMPORTE', 'median')
    ).sort_values(by=['AÑO_FOPE','MES_FOPE'], ascending=True)
)

In [16]:
agrupado_operaciones_anuales_mes.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales_mes.csv"), index=False)